In [14]:
import requests
import csv
from datetime import datetime, timedelta

def get_kalshi_contract_expirations():
    base_url = "https://demo-api.kalshi.co/trade-api/v2"
    now = datetime.utcnow()
    cutoff = now + timedelta(days=90)
    cutoff_str = cutoff.strftime("%Y-%m-%dT%H:%M:%SZ")

    # 1) fetch open markets expiring soon
    markets_resp = requests.get(
        f"{base_url}/markets",
        params={"status": "open", "max_close_time": cutoff_str},
        timeout=10
    )
    markets_resp.raise_for_status()

    all_markets = markets_resp.json().get("markets", [])
    # 2) drop any without a valid id
    valid_markets = [m for m in all_markets if m.get("id")]
    print(f"Found {len(all_markets)} markets; {len(valid_markets)} have valid IDs")

    results = []

    for market in valid_markets:
        market_id   = market["id"]
        market_name = market.get("title", "Unknown")

        # parse close_time
        close_str = market.get("close_time", "")
        try:
            if "." in close_str:
                close_dt = datetime.strptime(close_str, "%Y-%m-%dT%H:%M:%S.%fZ")
            else:
                close_dt = datetime.strptime(close_str, "%Y-%m-%dT%H:%M:%SZ")
        except Exception:
            print(f"  ❗ couldn't parse close_time {close_str!r} for {market_name}")
            continue

        delta = close_dt - now
        days = delta.days
        hours = delta.seconds // 3600
        time_remaining = f"{days} days, {hours} hours"

        # fetch that market’s contracts
        url = f"{base_url}/markets/{market_id}/contracts"
        ctr_resp = requests.get(url, timeout=10)
        if ctr_resp.status_code != 200:
            print(f"  ⚠️ skipping {market_id!r} – got {ctr_resp.status_code}")
            continue

        contracts = ctr_resp.json().get("contracts", [])
        for c in contracts:
            results.append({
                "Market Name": market_name,
                "Contract ID":  c.get("id"),
                "Time to Resolution": time_remaining
            })

    # write CSV
    with open("contract_expirations.csv", "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["Market Name","Contract ID","Time to Resolution"])
        writer.writeheader()
        writer.writerows(results)

    return results

if __name__ == "__main__":
    print("Fetching Kalshi contracts expiring within 90 days…")
    data = get_kalshi_contract_expirations()
    print(f"  → wrote {len(data)} rows to CSV")

Fetching Kalshi contracts expiring within 90 days…
Found 100 markets; 0 have valid IDs
  → wrote 0 rows to CSV
